# Réalisez une étude de santé publique

In [0]:
# Import des librairies
import numpy as np
import pandas as pd
import seaborn as sns

In [0]:
# Import des données des bilans alimentaires
url_ani = "https://raw.githubusercontent.com/xavierbarbier/openclassrooms/master/projet_3/data/FAOSTAT_data_animal_11-18-2019.csv"
url_veg ="https://raw.githubusercontent.com/xavierbarbier/openclassrooms/master/projet_3/data/FAOSTAT_data_vegetal_11-18-2019.csv"


ani = pd.read_csv(url_ani)
veg = pd.read_csv(url_veg)


In [0]:
# Ajout de la variable origin
ani["origin"] = "animal"
veg["origin"] = "vegetal"

# On regroupe veg et ani en un unique dataframe, via une union
temp = ani.append(veg)

# Suppression de ani et veg
del ani, veg

# On renomme les colonnes de temp
temp.columns = ["xx","xx2","country_code","country",'xx3','element'
    ,'item_code','item','xx4',"year","unit","value",'xx5','xx6'
    ,'origin']

# Transformation de temp en table pivot
data = temp.pivot_table(
    index=["country_code","country","item_code","item","year","origin"],
    columns = ["element"], values=["value"], aggfunc=sum)

data.columns = ['feed','other_uses', 'food_supply_kcalcapitaday','food_supply_quantity_kgcapitayr',
                "fat_supply_quantity_gcapitaday",'protein_supply_quantity_gcapitaday',
                'domestic_supply_quantity', 'export_quantity', 'import_quantity', 'food',  'waste',
                'production','seed', 'processing', 'stock_variation']
data = data.reset_index()
data.head()

,country_code,country,item_code,item,year,origin,feed,other_uses,food_supply_kcalcapitaday,food_supply_quantity_kgcapitayr,fat_supply_quantity_gcapitaday,protein_supply_quantity_gcapitaday,domestic_supply_quantity,export_quantity,import_quantity,food,waste,production,seed,processing,stock_variation
0,1,Arménie,2511,Blé,2013,vegetal,93.0,0.0,1024.0,130.60,3.60,30.52,554.0,1.0,361.0,389.0,32.0,312.0,30.0,10.0,-118.0
1,1,Arménie,2513,Orge,2013,vegetal,137.0,26.0,0.0,0.00,0.00,0.00,198.0,0.0,9.0,0.0,15.0,189.0,14.0,7.0,0.0
2,1,Arménie,2514,Maïs,2013,vegetal,96.0,NaN,0.0,0.03,NaN,0.01,102.0,NaN,82.0,0.0,7.0,21.0,0.0,NaN,NaN
3,1,Arménie,2515,Seigle,2013,vegetal,1.0,NaN,1.0,0.12,0.00,0.02,1.0,NaN,0.0,0.0,0.0,1.0,0.0,NaN,0.0
4,1,Arménie,2516,Avoine,2013,vegetal,4.0,NaN,2.0,0.37,0.03,0.09,6.0,NaN,1.0,1.0,0.0,5.0,0.0,NaN,NaN


In [0]:
#Importation données population
url_population = "https://raw.githubusercontent.com/xavierbarbier/openclassrooms/master/projet_3/data/FAOSTAT_data_population_11-18-2019.csv"

population = pd.read_csv(url_population)

## Question 1 : 
Calcul population mondiale pour l'année 2013

In [0]:
#Calcul population mondiale (en millards)
total = sum(population.Valeur)*1000
print(total) #7.210845847999999 millards

7210845847.999999


* la population mondiale en 2013 est estimée à 7.210845847999999 millards



In [0]:
#Merge data + population
bilan = pd.merge(data,population,left_on= "country_code", right_on= "Code zone")

## Question 3 :
Calculez (pour chaque pays et chaque produit) la disponibilité alimentaire en kcal puis en kg de protéines.

In [0]:
##calcul disponibilité alimentaire en kcal par an
bilan["dispo_kcal_year"] = bilan.food_supply_kcalcapitaday*365*bilan.Valeur*1000 

##Calcul dispo proteine par an en kg
bilan["dispo_prot_kg_year"]=(bilan.protein_supply_quantity_gcapitaday*365*bilan.Valeur)/1000  

## Question 4 : 
Calculez pour chaque produit le ratio "énergie/poids" en kcal/kg. Comparaison avec la valeur calorique d'un oeuf.

In [0]:
#QUESTION 4

## Ratio énergie/poids
bilan["ratio_kcal_per_kg"]=(bilan.food_supply_kcalcapitaday*365)/bilan.food_supply_quantity_kgcapitayr

## calcul moyenne pour vérifier + commentaire comparaison source
print(np.nanmean(bilan.ratio_kcal_per_kg[bilan.item == "Oeufs"]) )

1363.8628304280373


*   Oeuf = 1363.8628304280373 kcal/kg - Wikipedia = 1470kcal/kg






In [0]:
## calcul Ratio protéines/poids total 
bilan["ratio_prot"]= bilan.dispo_prot_kg_year/(bilan.food_supply_quantity_kgcapitayr*bilan.Valeur)

##moyenne pour vérifier + commentaire comparaison source  
print (np.nanmean(bilan.ratio_prot[bilan.item == "Avoine"]) )
###-> 0.07930095652715326 - 

0.07930095652715326



*   Avoine = 0.08 g prot/kg - Wikipedia = 0.01070



# Question 5 : 
Citez 5 aliments parmi les 20 aliments les plus caloriques, en utilisant le ratio énergie/poids.


In [0]:
#QUESTION 5
##Calories
caloriques = bilan.groupby(['item']).mean()
caloriques.sort_values(["ratio_kcal_per_kg"], axis=0, ascending=False, inplace=True) 
caloriques = caloriques.reset_index()
caloriques.head(20)[["item","ratio_kcal_per_kg"]]


,item,ratio_kcal_per_kg
0,Graines Colza/Moutarde,inf
1,Palmistes,inf
2,Huile de Palme,8791.492506
3,Huile de Soja,8621.385781
4,Huile de Son de Riz,8598.244664
5,Huile de Colza&Moutarde,8355.917634
6,Huile de Tournesol,8308.469692
7,Huil Plantes Oleif Autr,8229.895971
8,Huile Graines de Coton,8173.066992
9,Huile de Coco,8142.212217


Citez 5 aliments parmi les 20 aliments les plus riches en protéines.

In [0]:
##proteines
proteines = bilan.groupby(['item']).mean()
proteines.sort_values(["ratio_prot"], axis=0, ascending=False, inplace=True) 
proteines = proteines.reset_index()
proteines.head(20)[["item","ratio_prot"]]

,item,ratio_prot
0,Soja,0.252841
1,Arachides Decortiquees,0.247759
2,Pois,0.221286
3,Légumineuses Autres,0.220572
4,Haricots,0.217400
5,Sésame,0.178310
6,Abats Comestible,0.176206
7,"Viande, Autre",0.173685
8,"Plantes Oleiferes, Autre",0.169097
9,Aliments pour enfants,0.148697


# Question 6 : 
Calculez, pour les produits végétaux uniquement, la disponibilité intérieure mondiale exprimée en kcal.

In [0]:
#QUESTION 6
##Selection origine Vegetal pour création bilan vegetal 
bilan_vegetal = bilan[bilan.origin == "vegetal"]

## Création variable Dispo int Kcal
bilan_vegetal["dispo_int_kcal"] = bilan_vegetal.ratio_kcal_per_kg*bilan_vegetal.domestic_supply_quantity*1000*1000

bilan_vegetal_nan = bilan_vegetal.dropna(subset = ['dispo_int_kcal'])
np.sum(bilan_vegetal_nan.dispo_int_kcal)

dispo_int_mond_vege = np.sum(bilan_vegetal_nan.dispo_int_kcal[np.isfinite(bilan_vegetal_nan.dispo_int_kcal)]) # 1.216060325067501e+16 kcal
print(dispo_int_mond_vege)

1.216060325067501e+16


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


* Disponiblité intérieure mondiale végétal = 1.216060325067501e+16 kcal

# Question 7 : 
Combien d'humains pourraient être nourris si toute la disponibilité intérieure mondiale de produits végétaux était utilisée pour de la nourriture ? Donnez les résultats en termes de calories, puis de protéines, et exprimez ensuite ces 2 résultats en pourcentage de la population mondiale.

In [0]:
#QUESTION 7     
## Nombre humain apport minimum kcal via vegetal
print((dispo_int_mond_vege/(2800*365))/1000000000) #2800 kcal/jours
print(((dispo_int_mond_vege/(2800*365))/total)*100) #-> 165%

11.898829012402162
165.01294387956474


* Il serait possible de nourrir 11.9 milliards d'humains, soit 165% de la population mondiale uniquement avec les produits végétaux.

In [0]:
##Dispo intérieure proteine en kg
bilan_vegetal["dispo_int_prot_kg"] = (bilan_vegetal.domestic_supply_quantity*1000*1000)*bilan_vegetal.ratio_prot 

bilan_vegetal_nan = bilan_vegetal.dropna(subset = ['dispo_int_prot_kg'])

dispo_int_mond_prot =  np.sum(bilan_vegetal_nan.dispo_int_prot_kg[np.isfinite(bilan_vegetal_nan.dispo_int_prot_kg)]) # 1.216060325067501e+16 kcal

print((dispo_int_mond_prot/(0.066*365))/1000000000) #66 g/jours
print(((dispo_int_mond_prot/(0.066*365))/total)*100) #-> 167%

12.109818600711266
167.93894719119598


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


* Il serait possible de founir a 12.1 milliards d'humains, soit 167 % de la population mondiale, un apport en proteine minimal uniquement avec les produits végétaux.

# Question 8 : 
Combien d'humains pourraient être nourris si toute la disponibilité alimentaire en produits végétaux (Food) la nourriture végétale destinée aux animaux (Feed) et les pertes de produits végétaux (Waste) étaient utilisés pour de la nourriture ?

In [0]:
#Question 8

## Calcul disponibilité végétale (nourriture + feed + pertes)
bilan_vegetal["dispo_veg"]=bilan_vegetal.food + bilan_vegetal.feed + bilan_vegetal.waste

## Calcul disponibilité végétale Kcal
bilan_vegetal["dispo_veg_kcal"]=bilan_vegetal.ratio_kcal_per_kg*((bilan_vegetal.dispo_veg)*1000*1000)

## Calcul disponibilité végétale kcal totale
bilan_vegetal_nan = bilan_vegetal.dropna(subset = ['dispo_veg_kcal'])

dispo_vege_monde =  np.sum(bilan_vegetal_nan.dispo_veg_kcal[np.isfinite(bilan_vegetal_nan.dispo_veg_kcal)])

##Nombre de personnes + ratio
print((dispo_vege_monde/(2800*365))/1000000000) #2800 kcal/jours -> 6 milliards
print(((dispo_vege_monde/(2800*365))/total)*100) #-> 83%

6.021853376698411
83.5110540931704


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


* Avec la nourriture pour animaux et le pertes, il serait possible de nourrir 6 milliards d'humains, soit 83% de la population mondiale uniquement avec les produits végétaux.

In [0]:
## disponibilité végétale totale proteine
bilan_vegetal["dispo_vege_prot_kg"] = (bilan_vegetal.dispo_veg*1000*1000)*bilan_vegetal.ratio_prot

bilan_vegetal_nan = bilan_vegetal.dropna(subset = ['dispo_vege_prot_kg'])

dispo_vegetal_prot =  np.sum(bilan_vegetal_nan.dispo_vege_prot_kg[np.isfinite(bilan_vegetal_nan.dispo_vege_prot_kg)])

##Nombre de personnes + ratio
print((dispo_vegetal_prot/(0.066*365))/1000000000) #66 g/jours -> 8,4 milliards
print(((dispo_vegetal_prot/(0.066*365))/total)*100) #-> 92 %

6.700498049291322
92.92249745083335


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


* Avec la nourriture pour animaux et le pertes, il serait possible de fournir à 6.7 milliards d'humains, soit 92% de la population mondiale, un apport en proteine minimal uniquement avec les produits végétaux.

# Question 9 : 
Combien d'humains pourraient être nourris avec la disponibilité alimentaire mondiale ?

In [0]:
#Question 9

## Calcul disponibilité totale (int + végétale) + ratio
bilan_vegetal_nan = bilan_vegetal.dropna(subset = ['dispo_kcal_year'])
pop_vege = np.sum(bilan_vegetal.dispo_kcal_year[np.isfinite(bilan_vegetal.dispo_kcal_year)])/(2800*365)

print(pop_vege/total*100) #-> 82%

82.65430882071229


In [0]:
## disponibilité totale proteine + ratio
bilan_vegetal_nan = bilan_vegetal.dropna(subset = ['dispo_vege_prot_kg'])
prot_vege = (np.sum(bilan_vegetal.dispo_vege_prot_kg[np.isfinite(bilan_vegetal.dispo_vege_prot_kg)]))/(0.066*365) #8,6 Milliards
print(prot_vege/total*100) #-> 92%

92.92249745083335


* Avec uniquement la disponibilité alimentaire végétale mondiale, il serait possible de nourrir à 82% de la population mondiale, et de fournir un apport en proteine minimal à 92% de la population mondiale.

# Question 10 : 
Quelle proportion de la population mondiale est considérée comme étant en sous-nutrition ?

In [0]:
#Question 10
##Import des données
url_malnutrition = "https://raw.githubusercontent.com/xavierbarbier/openclassrooms/master/projet_3/data/FAOSTAT_data_malnutrition_11-18-2019.csv"


malnutrition = pd.read_csv(url_malnutrition)

sousnutrition = malnutrition.pivot_table(
    index=["Zone","Code Élément"],
    columns = ["Année"], values="Valeur",aggfunc=sum)

sousnutrition = sousnutrition.reset_index()
sousnutrition.head(5)

Année,Zone,Code Élément,2012-2014,2013-2015,2014-2016,2015-2017,2016-2018
0,Afghanistan,6132,7.9,8.8,9.6,10.2,10.6
1,Afrique du Sud,6132,2.6,2.8,3.2,3.4,3.5
2,Albanie,6132,0.2,0.2,0.2,0.2,0.2
3,Algérie,6132,1.7,1.6,1.6,1.6,1.6
4,Allemagne,6132,0,0,0,0,0


In [0]:
##Nombre personnes sous nutrition en 2013
total_sousnutrition_2013 = np.sum(pd.to_numeric((sousnutrition["2012-2014"]), errors='coerce'))
print(total_sousnutrition_2013) #743.8000000000001 millions

## ratio
ratio_sousnutrition_2013 = ((total_sousnutrition_2013*1000000)/total)*100 # = 10,22%
print(ratio_sousnutrition_2013) # 10.3 %

743.8000000000001
10.315017345798628


* en 2013, 743 millions de personnes étaient considérées en sous-nutrition, soit 10.3% de la popuplation mondiale.

# Question 11 : 
En ne prenant en compte que les céréales destinées à l'alimentation (humaine et animale), quelle proportion (en termes de poids) est destinée à l'alimentation animale ?

In [0]:
#Question 11
##Import des données
url_crl = "https://raw.githubusercontent.com/xavierbarbier/openclassrooms/master/projet_3/data/FAOSTAT_data_crl_11-18-2019.csv"

crl = pd.read_csv(url_crl)

##création variable is_cereal
bilan["is_cereal"] = False

crl.rename(columns={'Code Produit':'code_produit'}, inplace=True)

## Liste des codes produits crl
list_crl = crl.code_produit.unique()
list_crl
##boucle pour booleen is_cereal

is_cereal = []

for item_code in bilan['item_code']:
    if item_code in list_crl:
        is_cereal.append(True)
    else:
        is_cereal.append(False)

bilan['is_cereal'] = is_cereal

## total 
bilan_crl = bilan[bilan["is_cereal"] == True]
feed_crl = bilan_crl["feed"].sum()/(bilan_crl["feed"].sum()  + bilan_crl["food"].sum())*100
print(round(feed_crl,0)) ### -> 46%

46.0


* 46% de la production mondiale de céréale est destiné à l'alimentation animale.

# Question 12 :
A partir des ratios suivant : 
* le ratio entre la quantité destinés aux "Autres utilisations" (Other uses) et la disponibilité intérieure.
* le ratio entre la quantité destinée à la nourriture animale et la quantité destinée à la nourriture (animale + humaine)

Donnez les 3 produits qui ont la plus grande valeur pour chacun des 2 ratios.

In [0]:
# QUESTION 12

## liste pays sous nutrition
sousnutrition_2014 = sousnutrition[sousnutrition['2013-2015'] != 0]
list_sousnutrition = sousnutrition_2014.Zone.unique()

## création variable sous nutrition
bilan["sous_nutrition"] = False

sous_nutrition = []

for country in bilan['country']:
    if country in list_sousnutrition:
        sous_nutrition.append(True)
    else:
        sous_nutrition.append(False)

bilan['sous_nutrition'] = sous_nutrition

## regroupement pays sous nutrition dans un data frame
bilan_pays_sousnutrition = bilan[bilan["sous_nutrition"] == True]

## Aggregations valeurs par item
group = bilan_pays_sousnutrition.groupby('item').sum()
group.sort_values(["export_quantity"], axis=0, ascending=False, inplace=True) 
group = group.reset_index()

## Selection top 15 produits
top15 = group.loc[0:15,]
list_top15 = top15.item.unique()

most_exported_products = bilan.loc[data['item'].isin(list_top15)]

most_exported_products.sort_values(["import_quantity"], axis=0, ascending=False, inplace=True)
most_exported_products = most_exported_products.reset_index()
top_importations = most_exported_products.loc[0:200,]
top_importations = top_importations.groupby('item').sum()

top_importations.sort_values(["import_quantity"], axis=0, ascending=False, inplace=True)

top_importations["ratio_other_domestic"] = top_importations.other_uses/top_importations.domestic_supply_quantity
top_importations["ratio_feed_total"] = top_importations.feed/(top_importations.feed+top_importations.food)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Ratio autres utilisation / disponibilité intérieure

In [0]:
top_importations.sort_values(["ratio_other_domestic"], axis=0, ascending=False).head(3)

,index,country_code,item_code,year,feed,other_uses,food_supply_kcalcapitaday,food_supply_quantity_kgcapitayr,fat_supply_quantity_gcapitaday,protein_supply_quantity_gcapitaday,domestic_supply_quantity,export_quantity,import_quantity,food,waste,production,seed,processing,stock_variation,Code zone,Code Élément,Code Produit,Code année,Année,Valeur,dispo_kcal_year,dispo_prot_kg_year,ratio_kcal_per_kg,ratio_prot,is_cereal,sous_nutrition,ratio_other_domestic,ratio_feed_total
item,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Huile de Palme,87149,1532,25770,20130,0.0,18726.0,756.0,32.09,85.43,0.23,25865.0,22919.0,30034.0,6922.0,224.0,19446.0,0.0,3.0,-698.0,1532,5110,30100,20130,20130,3598820.126,6.052143e+13,8.598549e+03,69696.325292,0.018109,0.0,5.0,0.723990,0.000000
Manioc,41454,785,10128,8052,25153.0,20355.0,178.0,61.96,0.36,0.89,67341.0,26237.0,34751.0,15268.0,4759.0,58765.0,0.0,1804.0,63.0,785,2044,12040,8052,8052,1793266.604,1.624320e+13,9.378766e+04,3313.731664,0.019078,0.0,3.0,0.302268,0.622276
Maïs,231992,3932,65364,52338,392164.0,181869.0,4539.0,563.99,39.82,107.70,687821.0,29315.0,101454.0,55283.0,17818.0,661019.0,3132.0,37559.0,-45340.0,3932,13286,78260,52338,52338,3174688.629,1.608186e+14,3.771737e+06,74054.438320,1.647570,26.0,14.0,0.264413,0.876448


Ratio autres nourriture animale / disponibilité alimentaire

In [0]:
top_importations.sort_values(["ratio_feed_total"], axis=0, ascending=False).head(3)

,index,country_code,item_code,year,feed,other_uses,food_supply_kcalcapitaday,food_supply_quantity_kgcapitayr,fat_supply_quantity_gcapitaday,protein_supply_quantity_gcapitaday,domestic_supply_quantity,export_quantity,import_quantity,food,waste,production,seed,processing,stock_variation,Code zone,Code Élément,Code Produit,Code année,Année,Valeur,dispo_kcal_year,dispo_prot_kg_year,ratio_kcal_per_kg,ratio_prot,is_cereal,sous_nutrition,ratio_other_domestic,ratio_feed_total
item,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Orge,63743,1160,17591,14091,20160.0,12.0,21.0,3.31,0.10,0.6,29600.0,6271.0,22515.0,365.0,953.0,13149.0,319.0,7790.0,206.0,1160,3577,21070,14091,14091,1891545.106,8.865076e+11,2.683842e+04,12769.028199,0.374109,7.0,2.0,0.000405,0.982217
Maïs,231992,3932,65364,52338,392164.0,181869.0,4539.0,563.99,39.82,107.7,687821.0,29315.0,101454.0,55283.0,17818.0,661019.0,3132.0,37559.0,-45340.0,3932,13286,78260,52338,52338,3174688.629,1.608186e+14,3.771737e+06,74054.438320,1.647570,26.0,14.0,0.264413,0.876448
Poissons Pelagiques,60044,1077,16578,12078,13097.0,40.0,115.0,36.16,5.02,15.7,17190.0,5115.0,14371.0,4052.0,0.0,7764.0,0.0,0.0,172.0,1077,3066,18060,12078,12078,1877449.413,4.639826e+12,6.417942e+05,6356.205204,0.836129,0.0,3.0,0.002327,0.763718


#QUESTION 13
Combien de tonnes de céréales pourraient être libérées si les USA diminuaient leur production de produits animaux de 10% ?

In [0]:
usa_crl = bilan_crl[bilan_crl.country == "États-Unis d'Amérique"]
np.sum(usa_crl.feed)*0.1 

14009.6

### -> 14009.6 milliers de tonnes pourraient être libérées si les USA diminuaient leur production de produits animaux de 10%





# QUESTION 14
* En Thaïlande, quelle proportion de manioc est exportée ? 
* Quelle est la proportion de personnes en sous-nutrition?


In [0]:
thailande = bilan[bilan.country == 'Thaïlande']
thailande = thailande[thailande.item == "Manioc"]

print(thailande.export_quantity/thailande.production*100)

12967    83.41273
dtype: float64




*  83.41273 %  manioc à l'exportation




In [0]:
total_thailande = population[population.Zone == "Thaïlande"]
total_thailande = total_thailande.Valeur
total_thailande

sousnutrition_thailande = sousnutrition[sousnutrition.Zone == "Thaïlande"]
sousnutrition_thailande = sousnutrition_thailande["2012-2014"]
sousnutrition_thailande = pd.to_numeric(sousnutrition_thailande)*1000
sousnutrition_thailande 
5600.0/68144.518*100


8.217829055596226

8 % de la population de la Thailaide est considérée en sous-nutrition.